In [1]:
# PennyLane imports
import pennylane as qml
from pennylane import numpy as pnp

from scipy.optimize import differential_evolution, minimize
from scipy.stats.qmc import Halton

# General imports
import os
import json
import numpy as np
from datetime import datetime

from qiskit.quantum_info import SparsePauliOp

# custom module
from susy_qm import calculate_Hamiltonian, create_vqe_plots

In [ ]:
#potential = 'QHO'
#potential = 'AHO'
potential = 'DW'
cut_offs_list = [16]
shots = 1024
shots = 5000


starttime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
folder = str(starttime)

#Create directory for files
#base_path = r"C:\Users\johnkerf\Desktop\Quantum-Computing\Quantum-Computing\SUSY\SUSY QM\PennyLane\SUSY VQE\TestFiles\{}\\{}\\".format(potential, folder)
base_path = r"C:\Users\Johnk\OneDrive\Desktop\PhD 2024\Quantum Computing Code\Quantum-Computing\SUSY\SUSY QM\PennyLane\SUSY VQE\TestFiles\{}\\{}\\".format(potential, folder)
os.makedirs(base_path)

print(f"Running for {potential} potential")

for cut_off in cut_offs_list:

    print(f"Running for cutoff: {cut_off}")

    #calculate Hamiltonian and expected eigenvalues
    H = calculate_Hamiltonian(cut_off, potential)
    eigenvalues = np.sort(np.linalg.eig(H)[0])
    min_eigenvalue = min(eigenvalues.real)

    #create qiskit Hamiltonian Pauli string
    hamiltonian = SparsePauliOp.from_operator(H)
    num_qubits = hamiltonian.num_qubits


    # Device
    shots = shots
    dev = qml.device('lightning.qubit', wires=num_qubits, shots=shots)


    #Initial params shape
    num_layers = 1
    params_shape = qml.StronglyEntanglingLayers.shape(n_layers=num_layers, n_wires=num_qubits)


    # Define the cost function
    @qml.qnode(dev)
    def cost_function(params):
        params = pnp.tensor(params.reshape(params_shape), requires_grad=True)
        qml.StronglyEntanglingLayers(weights=params, wires=range(num_qubits), imprimitive=qml.CZ)
        return qml.expval(qml.Hermitian(H, wires=range(num_qubits)))
    
    
    # VQE
    vqe_start = datetime.now()

    #variables
    num_vqe_runs = 1
    max_iterations = 10000
    strategy = 'rand1exp'
    popsize = 15
    tol = 0
    abs_tol = 1e-5

    #data arrays
    energies = []
    e2 = []
    x_values = []
    success = []
    run_times = []
    num_iters = []
    num_evaluations = []

    #Optimizer
    bounds = [(0, 2 * np.pi) for _ in range(np.prod(params_shape))]

    # Number of dimensions and population size
    num_dimensions = np.prod(params_shape)
    num_samples = popsize

    # Generate Halton sequence
    halton_sampler = Halton(d=num_dimensions, scramble=True)
    halton_samples = halton_sampler.random(n=num_samples)

    # Scale samples to parameter bounds (0 to 2*pi)
    scaled_samples = 2 * np.pi * halton_samples

    for i in range(num_vqe_runs):

        run_start = datetime.now()

        if i % 1 == 0:
            print(f"Run: {i}")

        
        # Differential Evolution optimization
        res = differential_evolution(cost_function, 
                                        bounds, 
                                        maxiter=max_iterations, 
                                        tol=tol,
                                        atol=abs_tol,
                                        strategy=strategy, 
                                        popsize=popsize,
                                        init=scaled_samples)
                                        
        
        if res.success == False:
            print("Not converged")

        #l_bfgs_b_result = minimize(
        #    fun=cost_function,      
        #    x0=res.x,              
        #    method = "COBYLA",
        #    bounds=bounds,   
        #    options={'maxiter':10000}
        #)

        #e2.append(l_bfgs_b_result.fun)
        energies.append(res.fun)
        x_values.append(res.x)
        success.append(res.success)
        num_iters.append(res.nit)
        num_evaluations.append(res.nfev)

        run_end = datetime.now()
        run_time = run_end - run_start
        run_times.append(run_time)

    vqe_end = datetime.now()
    vqe_time = vqe_end - vqe_start


Running for DW potential
Running for cutoff: 16
Run: 0


KeyboardInterrupt: 

In [44]:
e2

[]

In [45]:
energies

[np.float64(0.90659725541693)]

In [ ]:
8916409457647574

In [6]:
[x.real.tolist() for x in eigenvalues]

[0.8915993623272453,
 0.8916409457647909,
 2.734122249485066,
 2.734127815674369,
 4.843216012251254,
 4.8474766887186576,
 7.163335365454538,
 7.169032833441687,
 9.405161290351392,
 9.600102124552938,
 11.164258420998955,
 11.399970087059746,
 12.992367660453905,
 14.460674675713312,
 16.68212253567579,
 18.65066100732339,
 25.152443146529702,
 29.872395575641224,
 32.29950005993888,
 37.51424842448991,
 51.331545240046324,
 57.39518387591302,
 72.46260912341677,
 79.18396009845593,
 101.54235116796468,
 108.88764405352728,
 164.53174804457672,
 172.90234969440692,
 195.03378647731367,
 203.7668503684797,
 379.05819726427063,
 389.4353183097812]

In [ ]:
#Save run
run = {
    'potential': potential,
    'cutoff': cut_off,
    'exact_eigenvalues': [x.real.tolist() for x in eigenvalues],
    'ansatz': 'StronglyEntanglingLayers-1layer',
    'num_VQE': num_vqe_runs,
    'shots': shots,
    'Optimizer': {'name': 'differential_evolution',
                'bounds':'[(0, 2 * np.pi) for _ in range(np.prod(params_shape))]',
                'maxiter':max_iterations,
                'tolerance': tol,
                'strategy': strategy,
                'popsize': popsize
                },
    'results': energies,
    'params': [x.tolist() for x in x_values],
    'num_iters': num_iters,
    'num_evaluations': num_evaluations,
    'success': np.array(success, dtype=bool).tolist(),
    'run_times': [str(x) for x in run_times],
    'total_run_time': str(vqe_time)
}

# Save the variable to a JSON file
#path = r"C:\Users\Johnk\OneDrive\Desktop\PhD 2024\Quantum Computing Code\Quantum-Computing\SUSY\PennyLane\SUSY VQE\Shot Noise\Files\{}\\{}\{}_{}.json".format(potential, folder, potential, cut_off)
path = base_path + "{}_{}.json".format(potential, cut_off)
with open(path, 'w') as json_file:
    json.dump(run, json_file, indent=4)


In [ ]:
#base_path = r"C:\Users\Johnk\OneDrive\Desktop\PhD 2024\Quantum Computing Code\Quantum-Computing\SUSY\PennyLane\SUSY VQE\Shot Noise\Files\{}\\{}\\"
#create_vqe_plots(potential=potential, base_path=base_path, folder=folder, cut_off_list=cut_offs_list)